<a href="https://colab.research.google.com/github/Dobby233Liu/colab-test-notebooks/blob/master/DeepDanbooru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepDanbooru (pretrained) image-evaluating notebook for Colab
This notebook will install and run DeepDanbooru. It not only lets you evaluate online without the Kanotype demo that's painfully slow and offline for a long time per a day, but also a way to try new models and/or codes.

---

# Setup

In [ ]:
# install package
!cat DeepDanbooru/setup.py && rm -rf ./DeepDanbooru
!git clone --depth=1 https://github.com/KichangKim/DeepDanbooru.git
!pip install -q ./DeepDanbooru

cat: DeepDanbooru/setup.py: No such file or directory
Cloning into 'DeepDanbooru'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 47 (delta 0), reused 25 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# download pretrained model
!mkdir model
!wget -q -O model/model.zip "https://github.com/KichangKim/DeepDanbooru/releases/download/v4-20200814-sgd-e30/deepdanbooru-v4-20200814-sgd-e30.zip" # older but smaller
!cd model && unzip model.zip
!rm model/model.zip

Archive:  model.zip
  inflating: categories.json         
  inflating: model-resnet_custom_v4.h5  
  inflating: project.json            
  inflating: tags.txt                
  inflating: tags_log.json           
  inflating: tags-character.txt      
  inflating: tags-general.txt        


In [ ]:
# define a useful function
import deepdanbooru as dd
 
model = dd.project.load_model_from_project("./model", compile_model=False)
tags = dd.project.load_tags_from_project('./model')
 
def _evaluate(image_path, threshold):
    final_tags = []
    for tag, score in dd.commands.evaluate_image(image_path, model, tags, threshold):
            final_tags.append({ "tag": tag, "score": score.item() })
 
    return final_tags

---
# Evaluate image

In [ ]:
image_path = input("Image path (WARNING: this accepts only one file):\n")
final_tags = _evaluate(image_path, 0.5)
print(f'Tag(s) for {image_path}\n')
for entry in final_tags:
      score = entry["score"]
      tag = entry["tag"]
      print(f'(~{score:05.3f}) {tag}')

# Evaluation server

In [ ]:
# download a part of my repo for the ui files
!cat colab-test-notebooks/README.md && rm -rf ./colab-test-notebooks
!git clone --depth=1 "https://github.com/Dobby233Liu/colab-test-notebooks.git"

cat: colab-test-notebooks/README.md: No such file or directory
Cloning into 'colab-test-notebooks'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [ ]:
!pip install -q flask; #==0.12.2
!cat flask-ngrok/setup.py && rm -rf ./flask-ngrok
# actually proxy_url helper
!git clone --depth=1 https://github.com/Dobby233Liu/flask-ngrok.git
!pip install -q ./flask-ngrok

cat: flask-ngrok/setup.py: No such file or directory
Cloning into 'flask-ngrok'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# server
import flask
from flask_ngrok import hook_proxy_helper
import werkzeug
import os, sys
 
app = flask.Flask(__name__)
hook_proxy_helper(app)
 
# a hack made for _evaluate return
def createHtmlTbodyByEv(arr):
    html = ''
    for i in arr:
        html = html + '\t\t\t\t\t<tr>\n'
        html = html + f'\t\t\t\t\t\t<td><a href="http://danbooru.donmai.us/posts?tags={i["tag"]}">{i["tag"]}</a></td>\n'
        html = html + f'\t\t\t\t\t\t<td><abbr title="{i["score"]}">{i["score"]:05.3f}</abbr></td>\n'
        html = html + '\t\t\t\t\t</tr>\n'
    return html
 
def generic_predict():
  imagefile = flask.request.files['image']
  filename = werkzeug.utils.secure_filename(imagefile.filename)
  imagefile.save(filename)
 
  try:
    final_tags = _evaluate(filename, 0.5)
  finally:
    os.remove(filename)
  return final_tags
 
def my_html_files(id):
  file = "colab-test-notebooks/ddwebui/{ui}.html".format(ui=id)
  return file
def rfile(fn):
  with open(fn, "r") as f:
    return f.read()
 
@app.route('/predict', methods = ['POST'])
def handle_request_predict():
  try:
    content = {
      "result": "failed",
      "tags": [],
      "ex": "???"
    }
    status = 500
    try:
      content["tags"] = generic_predict()
      content["result"] = "complete"
      status = 200
      content["ex"] = ""
    except Exception as e:
      content["ex"] = repr(e)
    return flask.jsonify(content), status
  except Exception as e:
    sys.stdout.write(repr(e) + "\n")
    raise
 
@app.route('/predict_html', methods = ['POST'])
def handle_request_predict_html():
    final_tags = createHtmlTbodyByEv(generic_predict())
 
    return flask.make_response(rfile(my_html_files("predict")).format(tableOfContents=final_tags))
 
@app.route('/', methods = ['GET'])
def handle_request_form():
    return flask.send_file(my_html_files("index"))
 
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Port exposed to address https://1fgb7osolns-496ff2e9c6d22116-5000-colab.googleusercontent.com/


127.0.0.1 - - [27/Jul/2021 07:34:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:34:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jul/2021 07:34:25] "POST /predict_html HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:34:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jul/2021 07:34:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:34:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jul/2021 07:34:48] "POST /predict_html HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:34:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jul/2021 07:35:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:35:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Jul/2021 07:35:16] "POST /predict_html HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2021 07:35:17] "GET /favicon.ico HTTP/1.1" 404 -
